In [1]:
import pandas as pd
import mlflow
import os

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from scripts.hpo import read_from_s3


mlflow.set_tracking_uri(os.getenv("MLFLOW_SITE_URL"))
mlflow.set_experiment("red-wine-quality-prediction")

FILENAME = "winequality-red"
S3_BUCKET_NAME = os.getenv("S3_BUCKET_NAME")
S3_ENDPOINT = os.getenv("MLFLOW_S3_ENDPOINT_URL")
SEED = 0
TARGET = "quality"
FEATURES = [
    "volatile acidity", "citric acid",
    "sulphates", "alcohol"
]

2023/08/12 19:48:49 INFO mlflow.tracking.fluent: Experiment with name 'red-wine-quality-prediction' does not exist. Creating a new experiment.


In [2]:
df = read_from_s3(bucket_name=S3_BUCKET_NAME, filename=FILENAME)
X_train, X_test, y_train, y_test = \
    train_test_split(df[FEATURES], df[TARGET], random_state=SEED)

In [3]:
def run_pipeline(pipeline):
    pipeline.fit(X_train, y_train)
    accuracy = accuracy_score(y_test, pipeline.predict(X_test))
    mlflow.log_metric("accuracy", accuracy)

In [4]:
def run_experiment(pipeline):
    with mlflow.start_run() as run:
        mlflow.autolog()
        run_pipeline(pipeline)

In [5]:
linear_svc_pipeline = make_pipeline(
    StandardScaler(),
    LinearSVC()
)

run_experiment(linear_svc_pipeline)

2023/08/12 19:48:55 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2023/08/12 19:48:55 WARNING mlflow.tracking.fluent: Exception raised while enabling autologging for sklearn: module 'sklearn.metrics' has no attribute 'SCORERS'
/opt/conda/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_di

In [6]:
svc_pipeline = make_pipeline(
    StandardScaler(),
    SVC()
)

run_experiment(svc_pipeline)

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
knn_pipeline = make_pipeline(
    StandardScaler(),
    KNeighborsClassifier()
)

run_experiment(knn_pipeline)

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
rfc_pipeline = make_pipeline(
    StandardScaler(),
    RandomForestClassifier()
)

run_experiment(rfc_pipeline)